# Cryptocurrency Fundamental Analysis Tool

In [15]:
import json
import requests
import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np

pn.extension(sizing_mode="stretch_width")

In [16]:

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2018-01-01").isoformat()
end_date = pd.Timestamp.today().isoformat()

# Number of records to retrieve
page_size = 6000

# Define the list of crypto assets to analyze
crypto_asset = ['btc', 'eth', 'doge', 'bnb']

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in crypto_asset:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     'Value': value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Change the value type to 'float' to allow graphing
    data_df['Value'] = data_df['Value'].astype(float)
    
    # Format the DataFrame to allow xgraphing
    updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    return updated_data_df


In [17]:
# Create a dataframe for each metric by calling the get_data function and passing the relevant Coin Metric reference
prices = get_data('ReferenceRate')
active_addresses = get_data('AdrActCnt')


#Rolling data doesn't work in plot below
prices['btc'].rolling(window=700).mean().hvplot()

:Curve   [Date]   (btc)

In [18]:
# Create price visualization widget using https://anel.holoviz.org/gallery/apis/stocks_hvplot.html

# QUESTIONS ???
# Why is the 'rolling function not working in the graph?
# How can I add a parameter to get_plot() with the metrics' dataframes as inputs?
# Can each graph have unique axis (not linked)?
# Why is "Token" selection not updating graphs?
# Can we incorporate the fundamental table on top?

def get_plot_prices(ticker, window_size):
    df = prices[ticker]
    return df.hvplot.line('Date', grid=True, responsive=True, height=300, framewise=False)

def get_plot_addresses(ticker, window_size):
    df = active_addresses[ticker]
    return df.hvplot.line('Date', grid=True, responsive=True, height=300, framewise=False)

In [19]:
# Creat a fancy dashboard

ticker = pn.widgets.Select(name='Tokens', options=crypto_asset)
window = pn.widgets.IntSlider(name='Window Size', value=5, start=1, end=21)

prices = pn.Row(
    pn.Column(title, ticker, window, width=300),
    get_plot_prices(ticker.options[0], window.value)
)

addresses = pn.Row(
    pn.Column(title, ticker, window, width=300),
    get_plot_addresses(ticker.options[0], window.value)
)

def update(event):
    prices[1].object = get_plot_prices(ticker.value, window.value)
    addresses[1].object = get_plot_addresses(ticker.value, window.value)


ticker.param.watch(update, 'value')
window.param.watch(update, 'value')


#pn.template.FastListTemplate(site="Dashboard", title="Crypto Fundamental Analysis Tool", sidebar=[ticker, window], main=[row[1]]).show()

template = pn.template.FastListTemplate(site="Dashboard", title="Crypto Fundamental Analysis Tool", theme='dark')
template.sidebar.append(ticker)
template.sidebar.append(window)

template.main.append(prices[1])
template.main.append(addresses[1])
template.show();

Launching server at http://localhost:60120


In [20]:
# To show side-by-side graphs: https://panel.holoviz.org/reference/templates/FastListTemplate.html